In [81]:
import numpy as np
import _pickle as cPickle
import gzip


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\piyus\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    return importlib.import_module(mname)
  File "C:\Users\piyus\Anaconda3\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 658, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 571, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 922, in create_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
ImportError: DLL load failed: The specified module could not be found.

During handling of the above exception, another exc

In [23]:
NUM_CLASSES = 10

In [24]:
learningRate  = 0.1
learningScale = 0.5

In [46]:
numEpochs = 5
errorFrequency = 10000

In [61]:
dataTrain, dataValidate, dataTest = cPickle.load(gzip.open(r'C:\Users\piyus\Downloads\Downloads1\mnist.pkl.gz', 'rb'),encoding="latin1")

In [77]:
# training data (x = data 784 x 50000, y = labels 50000 x 1)
xTrain = dataTrain[0].T
yTrain = dataTrain[1]
print (xTrain.shape)
print (yTrain.shape)

(784, 50000)
(50000,)


In [78]:
xValidate = dataValidate[0].T
yValidate = dataValidate[1]
print (xValidate.shape)
print (yValidate.shape)

(784, 10000)
(10000,)


In [79]:
xTest = dataTest[0].T
yTest = dataTest[1]
print (xTest.shape)
print (yTest.shape)

(784, 10000)
(10000,)


In [65]:
(Ktrain,    Ntrain)    = xTrain.shape
(Kvalidate, Nvalidate) = xValidate.shape
(Ktest,     Ntest)     = xTest.shape

In [66]:
# 0 matrix vector multiplication
K0 = Ktrain # 28*28 = 784
M0 = 100    # 10 features per class input to final linear layer

# 1 bias addition
K1 = M0
M1 = K1

# 2 ReLU
K2 = M1
M2 = K2

# 3 matrix vector multiplication
K3 = M2
M3 = NUM_CLASSES

# 4 bias addition
K4 = M3
M4 = K4


In [67]:
# weights Gaussian 0 mean 1 variance
H0 = np.random.randn(M0, K0)
h1 = np.random.randn(K1)
H3 = np.random.randn(M3, K3)
h4 = np.random.randn(K4)

# feature maps
x0 = np.zeros(K0)
x1 = np.zeros(K1)
x2 = np.zeros(K2)
x3 = np.zeros(K3)
x4 = np.zeros(K4)
x5 = np.zeros(M4)

# error calculations
px = np.zeros(M4)

# gradients
dedx1 = np.zeros(K1)
dedx2 = np.zeros(K2)
dedx3 = np.zeros(K3)
dedx4 = np.zeros(K4)
dedx5 = np.zeros(M4)

In [69]:
eNumVal = 0
for n in range(Nvalidate):

    # forward propagation
    x0 = np.copy(xValidate[:, n])
    x1 = np.dot(H0, x0)
    x2 = np.add(x1, h1)
    x3 = np.maximum(0, x2)
    x4 = np.dot(H3, x3)
    x5 = np.add(x4, h4)

    # track errors
    khat = np.argmax(x5)
    if yValidate[n] != khat:
        eNumVal = eNumVal + 1



In [70]:
# normalize the error percentage
eNumVal = (100.0*eNumVal)/Nvalidate

In [71]:
print
print("Validation initial accuracy:        {0:6.2f}".format(100.0 - eNumVal))
print


Validation initial accuracy:         92.36


<function print>

In [73]:

# error reset
eIndex = 0
eNum   = 0
eSum   = 0.0

# cycle through the epochs
for epoch in range(numEpochs):
    
    #
    # training
    #

    # cycle through the inputs
    for n in range(Ntrain):

        # forward propagation
        x0 = np.copy(xTrain[:, n])
        x1 = np.dot(H0, x0)
        x2 = np.add(x1, h1)
        x3 = np.maximum(0, x2)
        x4 = np.dot(H3, x3)
        x5 = np.add(x4, h4)

        # error calculation
        kstar        = yTrain[n]
        px           = np.subtract(x5, np.max(x5))    # softmax
        px           = np.exp(px)                     # softmax
        px           = np.true_divide(px, np.sum(px)) # softmax
        e            = -np.log(px[kstar])             # cross entropy
        dedx5        = np.copy(px)                    # initial error gradient
        dedx5[kstar] = dedx5[kstar] - 1.0             # initial error gradient

        # error percentage (local training data)
        khat = np.argmax(x5)
        if kstar != khat:
            eNum = eNum + 1
            
        # error sum (local training data)
        eSum = eSum + e
        
        # error reporting
        eIndex = eIndex + 1
        if eIndex == errorFrequency:
            eNum = (100.0*eNum)/errorFrequency
            eSum = (1.0*eSum)/errorFrequency
            print("Training local accuracy and error:  {0:6.2f}   {1:6.2f}".format(100.0 - eNum, eSum))
            eIndex = 0
            eNum   = 0
            eSum   = 0.0
        
        # backward propagation
        dedx4 = np.copy(dedx5)
        dedx3 = np.dot(H3.T, dedx4)
        dedx2 = np.multiply(np.minimum(1.0, np.ceil(np.maximum(0.0, x2))), dedx3)
        dedx1 = np.copy(dedx2)
        
        # weight gradient
        dedH0 = np.outer(dedx1, x0)
        dedh1 = np.copy(dedx2)
        dedH3 = np.outer(dedx4, x3)
        dedh4 = np.copy(dedx5)

        # weight update
        H0 = np.subtract(H0, np.multiply(learningRate, dedH0))
        h1 = np.subtract(h1, np.multiply(learningRate, dedh1))
        H3 = np.subtract(H3, np.multiply(learningRate, dedH3))
        h4 = np.subtract(h4, np.multiply(learningRate, dedh4))

    # initialize the error
    eNumVal = 0

    # cycle through the inputs
    for n in range(Nvalidate):

        # forward propagation
        x0 = np.copy(xValidate[:, n])
        x1 = np.dot(H0, x0)
        x2 = np.add(x1, h1)
        x3 = np.maximum(0, x2)
        x4 = np.dot(H3, x3)
        x5 = np.add(x4, h4)

        # track errors
        khat = np.argmax(x5)
        if yValidate[n] != khat:
            eNumVal = eNumVal + 1

    # normalize the error percentage
    eNumVal = (100.0*eNumVal)/Nvalidate

    # display results
    print
    print("Validation current accuracy:        {0:6.2f}".format(100.0 - eNumVal))
    print

    #
    # learning rate update
    #

    # update the learning rate
    learningRate = learningScale*learningRate


################################################################################
#
# Testing
#
################################################################################

# initialize the error
eNumTest = 0

# cycle through the inputs
for n in range(Ntest):

    # forward propagation
    x0 = np.copy(xTest[:, n])
    x1 = np.dot(H0, x0)
    x2 = np.add(x1, h1)
    x3 = np.maximum(0, x2)
    x4 = np.dot(H3, x3)
    x5 = np.add(x4, h4)

    # track errors
    khat = np.argmax(x5)
    if yTest[n] != khat:
        eNumTest = eNumTest + 1

# normalize the error percentage
eNumTest = (100.0*eNumTest)/Ntest

# display results
print("Testing accuracy:                   {0:6.2f}".format(100.0 - eNumTest))
print

Training local accuracy and error:   92.49     0.28
Training local accuracy and error:   92.41     0.28
Training local accuracy and error:   92.30     0.28
Training local accuracy and error:   92.16     0.29
Training local accuracy and error:   92.41     0.27
Validation current accuracy:         92.78
Training local accuracy and error:   92.70     0.27
Training local accuracy and error:   92.54     0.27
Training local accuracy and error:   92.56     0.27
Training local accuracy and error:   92.40     0.28
Training local accuracy and error:   92.61     0.26
Validation current accuracy:         92.86
Training local accuracy and error:   92.73     0.26
Training local accuracy and error:   92.72     0.27
Training local accuracy and error:   92.65     0.27
Training local accuracy and error:   92.52     0.27
Training local accuracy and error:   92.73     0.26
Validation current accuracy:         92.95
Training local accuracy and error:   92.76     0.26
Training local accuracy and error:   92

<function print>